# Bronze  
Data cleaning

In [1]:
import os
from main.utils import categorize_columns, capitalize_columns, columns_to_datetime
from pandas import merge
from pandas.core.interchange.dataframe_protocol import DataFrame, Column
from pyspark.examples.src.main.python.sql.arrow import dataframe_with_arrow_example
from pyspark.sql import SparkSession
import pandas as pd
import pyarrow
import fastparquet

In [ ]:
#park = SparkSession.builder.appName('Sales Forecast App').getOrCreate()

Reading files

In [ ]:

pyspark_file_list = []
files_location = '../raw_data'
#read the csv files in the location parameterized: '../raw_data'

read_my_data(files_location)
        
data_customers.dtypes

Checking data types

In [3]:
files_list = ['../raw_data/olist_customers_dataset.csv',
              '../raw_data/olist_order_items_dataset.csv',
              '../raw_data/olist_order_payments_dataset.csv',
              '../raw_data/olist_orders_dataset.csv',
              '../raw_data/olist_products_dataset.csv',
              '../raw_data/olist_sellers_dataset.csv',
              '../raw_data/product_category_name_translation.csv']
df_list = []

for file_path in files_list:
        # Extract a meaningful identifier from the file path
        # This splits the path on '/', takes the last part, and then removes the '.csv' extension
        identifier = file_path.split('/')[-1][:-4]

        # Convert identifier to a more uniform format if necessary
        identifier = identifier.replace('olist_', '').replace('_dataset', '')

        var_name = f'data_{identifier}'
        # Create a global variable with the name 'data_' followed by the identifier
        globals()[var_name] = pd.read_csv(file_path)
        
        df_list.append(var_name)
        # Display the DataFrame name that has been loaded
        print(f"Loaded new DataFrame: data_{identifier}")


Loaded new DataFrame: data_customers
Loaded new DataFrame: data_order_items
Loaded new DataFrame: data_order_payments
Loaded new DataFrame: data_orders
Loaded new DataFrame: data_products
Loaded new DataFrame: data_sellers
Loaded new DataFrame: data_product_category_name_translation


In [4]:
data_customers.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [20]:
for df in df_list:
    print(df.dtypes)

AttributeError: 'str' object has no attribute 'dtypes'

In [46]:
product_counts = data_products['product_id'].groupby(data_products['product_id']).count()
duplicated_product_ids = product_counts[product_counts > 1]
duplicated_products = data_products[data_products['product_id'].isin(duplicated_product_ids.index)]
print(duplicated_products)

Empty DataFrame
Columns: [product_id, product_category_name, product_name_lenght, product_description_lenght, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm]
Index: []


# Cleaning
Capitalize customer cities

In [5]:
capitalize_columns(dataframe=data_customers, col_name='customer_city')
capitalize_columns(data_sellers, 'seller_city')
#data_customers['customer_city'] = data_customers['customer_city'].str.title()
#data_sellers['seller_city']=data_sellers['seller_city'].str.title()

In [22]:
data_sellers['seller_city']

0                Campinas
1              Mogi Guacu
2          Rio De Janeiro
3               Sao Paulo
4       Braganca Paulista
              ...        
3090              Sarandi
3091              Palhoca
3092            Sao Paulo
3093              Pelotas
3094              Taubate
Name: seller_city, Length: 3095, dtype: object

In [ ]:
#pyspark_file_list

Parse date column

In [6]:
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
        
columns_to_datetime(data_orders, date_columns)

In [19]:
data_orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [23]:
data_order_payments.isna().sum()
#, , data_order_payments]

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

Change the datatype of Order status and Product category names columns from regular string to categorical variable¶


In [8]:
categorize_columns(data_orders, 'order_status')
categorize_columns(data_products,'product_category_name')
categorize_columns(data_product_category_name_translation, 'product_category_name')
categorize_columns(data_product_category_name_translation, 'product_category_name_english')



Join datasets

In [9]:
from functools import reduce
products = pd.merge(left = data_products, right = data_product_category_name_translation, how = 'left', left_on= data_products.product_category_name, right_on= data_product_category_name_translation.product_category_name).drop(['product_category_name_x', 'key_0'], axis = 1)
products = products[['product_id', 'product_category_name_y', 'product_category_name_english']].rename(columns={'product_category_name_y': 'original_category_name','product_category_name_english':'english_category_name'})
products.count()

product_id                32951
original_category_name    32328
english_category_name     32328
dtype: int64

In [54]:
orders_assist = [data_orders, data_order_items, data_order_payments]
#merge, join multiple datasets with the same FK:
orders_df = reduce(lambda data1, data2: pd.merge(data1, data2, on='order_id', how='inner'), orders_assist)

In [55]:
orders_with_sellers_df = pd.merge(left=orders_df, right=data_sellers, on='seller_id', how='left')

In [56]:
with_customers_df = pd.merge(left=orders_with_sellers_df, right=data_customers, on='customer_id', how='left')

Join with products and remove where product_id is null

I can drop those orders where we don't have same order_id in data_order_items. These are orders that are unavailable or cancelled

In [53]:
temp_order = pd.merge(left = data_orders, right=data_order_items, on= 'order_id', how='left')
temp_order[temp_order['product_id'].isnull()]



,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
306,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaT,NaT,2017-12-05,NaN,NaN,NaN,NaN,NaN,NaN
671,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,NaT,NaT,2018-02-16,NaN,NaN,NaN,NaN,NaN,NaN
791,37553832a3a89c9b2db59701c357ca67,7607cd563696c27ede287e515812d528,unavailable,2017-08-14 17:38:02,2017-08-17 00:15:18,NaT,NaT,2017-09-05,NaN,NaN,NaN,NaN,NaN,NaN
850,d57e15fb07fd180f06ab3926b39edcd2,470b93b3f1cde85550fc74cd3a476c78,unavailable,2018-01-08 19:39:03,2018-01-09 07:26:08,NaT,NaT,2018-02-06,NaN,NaN,NaN,NaN,NaN,NaN
1294,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2018-08-28 15:26:39,NaT,NaT,NaT,2018-09-12,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113204,aaab15da689073f8f9aa978a390a69d1,df20748206e4b865b2f14a5eabbfcf34,unavailable,2018-01-16 14:27:59,2018-01-17 03:37:34,NaT,NaT,2018-02-06,NaN,NaN,NaN,NaN,NaN,NaN
113236,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaT,NaT,NaT,2018-10-01,NaN,NaN,NaN,NaN,NaN,NaN
113324,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaT,NaT,NaT,2018-09-27,NaN,NaN,NaN,NaN,NaN,NaN
113325,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaT,NaT,2017-09-15,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
big_table = pd.merge(left=with_customers_df, right=products, on='product_id', how='inner').drop('order_status', axis=1).drop('customer_id', axis=1).drop('customer_unique_id', axis=1)



In [59]:
big_table['product_id'].isna().sum()

np.int64(0)

In [34]:
big_table['english_category_name'].groupby(big_table['english_category_name']).count()

C:\Users\gabor_rabai\AppData\Local\Temp\ipykernel_30240\2260815815.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  big_table['english_category_name'].groupby(big_table['english_category_name']).count()


english_category_name
health_beauty                 9975
computers_accessories         8082
auto                          4379
bed_bath_table               11823
furniture_decor               8744
                             ...  
flowers                         33
arts_and_craftmanship           24
diapers_and_hygiene             39
fashion_childrens_clothes        8
security_and_services            2
Name: english_category_name, Length: 71, dtype: int64

In [ ]:
big_table['seller_zip_code_prefix']

In [13]:
big_table.to_parquet('../test_parquet/big_table_b1', partition_cols=['english_category_name')

In [ ]:
big_table.count(numeric_only=True)

In [ ]:
type_and_missing = pd.concat([data.isna().sum().sort_values(ascending = False), 
                              ((data.isna().sum()/data.isna().count())*100).sort_values(ascending=False), 
                              data.dtypes], axis=1, keys=['Total', 'Missing_%','Type'])
type_and_missing[type_and_missing['Total'] > 0]

In [ ]:
df = pd.DataFrame(data)

In [ ]:
display(df)